In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('/content/drive/MyDrive/indian_weather_dataset3.csv')
data.head()

,Temperature (°C),Humidity (%),Weather
0,21,99,Rainy
1,25,91,Rainy
2,22,91,Rainy
3,21,84,Rainy
4,37,32,Sunny


In [4]:
data.describe()

,Temperature (°C),Humidity (%)
count,503.000000,503.000000
mean,22.049702,63.415507
std,6.216505,19.987155
min,15.000000,30.000000
25%,17.000000,46.000000
50%,20.000000,64.000000
75%,26.000000,80.000000
max,39.000000,99.000000


In [5]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [6]:
print(data['Weather'].unique())

['Rainy' 'Sunny' 'cold' 'Partly Cloudy' 'Cloudy']


In [7]:
label_encoder = LabelEncoder()
data['Weather_Label'] = label_encoder.fit_transform(data['Weather'])

In [8]:
print(data['Weather_Label'].unique())

[2 3 4 1 0]


In [9]:
X = data[['Temperature (°C)', 'Humidity (%)']].values
y = data['Weather_Label'].values

In [10]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')

])

In [18]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
11/11 [==============================] - 3s 31ms/step - loss: 1.5695 - accuracy: 0.1495 - val_loss: 1.4641 - val_accuracy: 0.6173
Epoch 2/50
11/11 [==============================] - 0s 8ms/step - loss: 1.4027 - accuracy: 0.6636 - val_loss: 1.3236 - val_accuracy: 0.6914
Epoch 3/50
11/11 [==============================] - 0s 7ms/step - loss: 1.2503 - accuracy: 0.7352 - val_loss: 1.1773 - val_accuracy: 0.7531
Epoch 4/50
11/11 [==============================] - 0s 8ms/step - loss: 1.0975 - accuracy: 0.7664 - val_loss: 1.0284 - val_accuracy: 0.7778
Epoch 5/50
11/11 [==============================] - 0s 7ms/step - loss: 0.9475 - accuracy: 0.7882 - val_loss: 0.8843 - val_accuracy: 0.7778
Epoch 6/50
11/11 [==============================] - 0s 6ms/step - loss: 0.8083 - accuracy: 0.7944 - val_loss: 0.7558 - val_accuracy: 0.7901
Epoch 7/50
11/11 [==============================] - 0s 6ms/step - loss: 0.6912 - accuracy: 0.7944 - val_loss: 0.6519 - val_accuracy: 0.8148
Epoch 8/50
11/11 [=

In [20]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")

4/4 [==============================] - 0s 6ms/step - loss: 0.1582 - accuracy: 0.9505
Test Accuracy: 0.9504950642585754


In [21]:
def predict_weather(temperature, humidity):
    # Normalize input
    scaled_input = scaler.transform([[temperature, humidity]])
    # Predict weather class
    predicted_class = np.argmax(model.predict(scaled_input))
    # Decode predicted class label
    predicted_weather = label_encoder.inverse_transform([predicted_class])[0]
    return predicted_weather

In [22]:
temperature = 25
humidity = 85
predicted = predict_weather(temperature, humidity)
print(f"Predicted Weather: {predicted}")

1/1 [==============================] - 0s 162ms/step
Predicted Weather: Rainy


In [23]:
from sklearn.metrics import classification_report
y_pred = np.argmax(model.predict(X_test), axis=1)
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)
print("Classification Report:")
print(report)

4/4 [==============================] - 0s 3ms/step
Classification Report:
               precision    recall  f1-score   support

       Cloudy       0.00      0.00      0.00         4
Partly Cloudy       1.00      1.00      1.00        14
        Rainy       0.96      0.96      0.96        27
        Sunny       1.00      1.00      1.00         6
         cold       0.93      1.00      0.96        50

     accuracy                           0.95       101
    macro avg       0.78      0.79      0.78       101
 weighted avg       0.91      0.95      0.93       101



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
model.save("/content/drive/MyDrive/Weather_prediction.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:
import json

from IPython.display import FileLink

# Link the files
FileLink("/content/drive/MyDrive/Weather_prediction.h5")

/content/drive/MyDrive/Weather_prediction.h5